# Recursive and FastForward Merges

`git lg` is an alias for `git log --graph --abbrev-commit --decorate --format=format:'%C(bold blue)%h%C(reset) - %C(bold green)(%ar)%C(reset) %C(white)%s%C(reset) %C(dim white)- %an%C(reset)%C(bold yellow)%d%C(reset)' --all`

It's a nice way of keeping track of multi-branch projects on the command line

## Fast Forward Merges
Fast forwarding is simply a feature of git where if the source branch contains only commits which are decendents of the tip of the destination branch, it will simply be applied by moving the head pointer to the tip of that branch

In [1]:
mkdir FastForwardMerge
cd FastForwardMerge
git init

Initialized empty Git repository in /Users/kate/Documents/Uni/CS480/Assignments/FastForwardMerge/.git/


In [2]:
cat << EOF > FF.txt
Pumpernickel
Plethora
Idiot
EOF

git add .
git commit -m "Initial Commit"

[master (root-commit) 8af5a57] Initial Commit
 1 file changed, 3 insertions(+)
 create mode 100644 FF.txt


In [3]:
git branch FF_Merge
git checkout FF_Merge

Switched to branch 'FF_Merge'


In [4]:
cat << EOF > FF.txt
Pumpernickel
Xanthan
Idiot
EOF

git add .
git commit -m "Changed a word"

[FF_Merge 4fb4574] Changed a word
 1 file changed, 1 insertion(+), 1 deletion(-)


In [5]:
cat << EOF > FF.txt
Tabby
Xanthan
Idiot
EOF

git add .
git commit -m "Changed another word"

[FF_Merge f612073] Changed another word
 1 file changed, 1 insertion(+), 1 deletion(-)


In [6]:
git checkout master

Switched to branch 'master'


In [7]:
git lg

* f612073 - (0 seconds ago) Changed another word - Addie Morrison (FF_Merge)
* 4fb4574 - (1 second ago) Changed a word - Addie Morrison
* 8af5a57 - (2 seconds ago) Initial Commit - Addie Morrison (HEAD -> master)

In [8]:
git merge FF_Merge

Updating 8af5a57..f612073
Fast-forward
 FF.txt | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [9]:
git lg

* f612073 - (0 seconds ago) Changed another word - Addie Morrison (HEAD -> master, FF_Merge)
* 4fb4574 - (1 second ago) Changed a word - Addie Morrison
* 8af5a57 - (2 seconds ago) Initial Commit - Addie Morrison

Alternatively, we can specify no-ff

This keeps the repository history (subjectively) cleaner by showing what happened where

In [10]:
git reset --hard HEAD~2

HEAD is now at 8af5a57 Initial Commit


In [11]:
git lg

* f612073 - (1 second ago) Changed another word - Addie Morrison (FF_Merge)
* 4fb4574 - (2 seconds ago) Changed a word - Addie Morrison
* 8af5a57 - (3 seconds ago) Initial Commit - Addie Morrison (HEAD -> master)

In [12]:
git merge FF_Merge --no-ff -m "Merge branch FastForward"

Merge made by the 'recursive' strategy.
 FF.txt | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [13]:
git lg

*   17934d3 - (0 seconds ago) Merge branch FastForward - Addie Morrison (HEAD -> master)
|\  
| * f612073 - (1 second ago) Changed another word - Addie Morrison (FF_Merge)
| * 4fb4574 - (2 seconds ago) Changed a word - Addie Morrison
|/  
* 8af5a57 - (3 seconds ago) Initial Commit - Addie Morrison

## Recursive Merges
This is the default merge strategy that git uses (for standard two-branch merges).

Essentially, recursive merges solve a whole series of headaches that arise from doing development on larger projects where parts of history need to be merged at different times to keep up with development.

To avoid an intdeterminant scenario where one ancestor would have to be selected as the basis for the merge, git recursively walks down possible trees to find any/all common ancestors between the two commits, and uses that tree as the basis for the merge

But, recursive merges work right so often. Let's demonstrate one of the few cases where they go wrong



In [14]:
cd ..
mkdir RecursiveMerge
cd RecursiveMerge
git init

Initialized empty Git repository in /Users/kate/Documents/Uni/CS480/Assignments/RecursiveMerge/.git/


In [15]:
cat << EOF > Recursive.txt
Reading
Sleep
Kitten
EOF

git add .
git commit -m "Base file"

[master (root-commit) d0adf6d] Base file
 1 file changed, 3 insertions(+)
 create mode 100644 Recursive.txt


In [16]:
git branch Branch

In [17]:
git checkout Branch

cat << EOF > Recursive.txt
Reading
Sleep
Kitten
Fairness
EOF

git add .
git commit -m "Appended a line"

Switched to branch 'Branch'
[Branch 88d438f] Appended a line
 1 file changed, 1 insertion(+)


In [18]:
git checkout master

cat << EOF > Recursive.txt
Reading
Cthulhu
Kitten
EOF

git add .
git commit -m "Changed a line"

Switched to branch 'master'
[master 85ebca8] Changed a line
 1 file changed, 1 insertion(+), 1 deletion(-)


In [19]:
git checkout Branch
git merge master --no-ff -m "Merge Master"

Switched to branch 'Branch'
Auto-merging Recursive.txt
Merge made by the 'recursive' strategy.
 Recursive.txt | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [20]:
git checkout master
git merge Branch --no-ff -m "Merge Branch"

Switched to branch 'master'
Merge made by the 'recursive' strategy.
 Recursive.txt | 1 +
 1 file changed, 1 insertion(+)


In [21]:
git lg

*   70eec8b - (0 seconds ago) Merge Branch - Addie Morrison (HEAD -> master)
|\  
| *   73c7428 - (0 seconds ago) Merge Master - Addie Morrison (Branch)
| |\  
| |/  
|/|   
* | 85ebca8 - (1 second ago) Changed a line - Addie Morrison
| * 88d438f - (1 second ago) Appended a line - Addie Morrison
|/  
* d0adf6d - (2 seconds ago) Base file - Addie Morrison

In [22]:
git checkout Branch
cat << EOF > Recursive.txt
Reading
Sleep
Kitten
Socialism
EOF

git add .
git commit -m "Changed the appended line"

Switched to branch 'Branch'
[Branch 20a261a] Changed the appended line
 1 file changed, 2 insertions(+), 2 deletions(-)


In [23]:
git checkout master
cat << EOF > Recursive.txt
Reading
Sleep
Kitten
EOF

git add .
git commit -m "Undid the change"

Switched to branch 'master'
[master fcfaf42] Undid the change
 1 file changed, 1 insertion(+), 2 deletions(-)


In [24]:
git lg

* 20a261a - (0 seconds ago) Changed the appended line - Addie Morrison (Branch)
| * fcfaf42 - (0 seconds ago) Undid the change - Addie Morrison (HEAD -> master)
| *   70eec8b - (1 second ago) Merge Branch - Addie Morrison
| |\  
| |/  
|/|   
* |   73c7428 - (1 second ago) Merge Master - Addie Morrison
|\ \  
| |/  
| * 85ebca8 - (2 seconds ago) Changed a line - Addie Morrison
* | 88d438f - (2 seconds ago) Appended a line - Addie Morrison
|/  
* d0adf6d - (3 seconds ago) Base file - Addie Morrison

In [25]:
git checkout master
git merge Branch --no-ff -m "Final Merge"

Already on 'master'
Auto-merging Recursive.txt
CONFLICT (content): Merge conflict in Recursive.txt
Automatic merge failed; fix conflicts and then commit the result.


: 1

In [26]:
cat Recursive.txt

Reading
Sleep
Kitten
<<<<<<< HEAD
Socialism
>>>>>>> Branch


While to a human it's perfectly obvious what the result should be

```
Reading
Cthulhu
Kitten
Socialism
```

The recursive merge's tree can't actually resolve it automatically!

## Bonus points: Octopus merge!
2cde51fbd0f3

Octopus merge is a simple strategy for merging more than two branches (for unrelated feature branches, for instance)

Note that all changesets must be unrelated for automatic resolution to succeed

In [27]:
cd ..
mkdir Octopus
cd Octopus
git init

Initialized empty Git repository in /Users/kate/Documents/Uni/CS480/Assignments/Octopus/.git/


In [28]:
cat << EOF > masterFile.txt
Contents of a file
EOF

git add .
git commit -m "Initial Commit"

[master (root-commit) 8a8fe90] Initial Commit
 1 file changed, 1 insertion(+)
 create mode 100644 masterFile.txt


In [29]:
git branch Feature1
git checkout Feature1

cat << EOF > Feature1.txt
Contents of Feature1
EOF

git add .
git commit -m "Feature 1"

Switched to branch 'Feature1'
[Feature1 8b7282d] Feature 1
 1 file changed, 1 insertion(+)
 create mode 100644 Feature1.txt


In [30]:
git checkout master
git branch Feature2
git checkout Feature2

cat << EOF > Feature2.txt
Contents of Feature2
EOF

git add .
git commit -m "Feature 2"

Switched to branch 'master'
Switched to branch 'Feature2'
[Feature2 2fc60d8] Feature 2
 1 file changed, 1 insertion(+)
 create mode 100644 Feature2.txt


In [31]:
git checkout master
git branch Feature3
git checkout Feature3

cat << EOF > Feature3.txt
Contents of Feature3
EOF

git add .
git commit -m "Feature 3"

Switched to branch 'master'
Switched to branch 'Feature3'
[Feature3 23b16dc] Feature 3
 1 file changed, 1 insertion(+)
 create mode 100644 Feature3.txt


In [32]:
git checkout master

cat << EOF > masterFile.txt
Contents of THE file
EOF

git add .
git commit -m "Changed MasterFile"

Switched to branch 'master'
[master cb5b2e2] Changed MasterFile
 1 file changed, 1 insertion(+), 1 deletion(-)


In [33]:
git lg

* 23b16dc - (1 second ago) Feature 3 - Addie Morrison (Feature3)
| * cb5b2e2 - (1 second ago) Changed MasterFile - Addie Morrison (HEAD -> master)
|/  
| * 2fc60d8 - (2 seconds ago) Feature 2 - Addie Morrison (Feature2)
|/  
| * 8b7282d - (3 seconds ago) Feature 1 - Addie Morrison (Feature1)
|/  
* 8a8fe90 - (4 seconds ago) Initial Commit - Addie Morrison

In [34]:
git merge Feature1 Feature2 Feature3 -m "Cephalopod merge"

Trying simple merge with Feature1
Trying simple merge with Feature2
Trying simple merge with Feature3
Merge made by the 'octopus' strategy.
 Feature1.txt | 1 +
 Feature2.txt | 1 +
 Feature3.txt | 1 +
 3 files changed, 3 insertions(+)
 create mode 100644 Feature1.txt
 create mode 100644 Feature2.txt
 create mode 100644 Feature3.txt


In [35]:
git lg

*---.   7eab7b1 - (0 seconds ago) Cephalopod merge - Addie Morrison (HEAD -> master)
|\ \ \  
| | | * 23b16dc - (1 second ago) Feature 3 - Addie Morrison (Feature3)
| | * | 2fc60d8 - (2 seconds ago) Feature 2 - Addie Morrison (Feature2)
| | |/  
| * | 8b7282d - (3 seconds ago) Feature 1 - Addie Morrison (Feature1)
| |/  
* | cb5b2e2 - (1 second ago) Changed MasterFile - Addie Morrison
|/  
* 8a8fe90 - (4 seconds ago) Initial Commit - Addie Morrison